In [2]:
!pip install keyboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.7 MB/s eta 0:00:00


In [11]:
!pip show keyboard

Name: keyboard
Version: 0.13.5
Summary: Hook and simulate keyboard events on Windows and Linux
Home-page: https://github.com/boppreh/keyboard
Author: BoppreH
Author-email: boppreh@gmail.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [16]:
import keyboard
import time
import threading

In [17]:
runningP = -1

# RN arrays of processes
RN = {
	0: [0, 0, 0, 0, 0],
	1: [0, 0, 0, 0, 0],
	2: [0, 0, 0, 0, 0],
	3: [0, 0, 0, 0, 0],
	4: [0, 0, 0, 0, 0]
}

In [18]:
token = {
	"token_owner": 2,
	"Q": [],
	"LN": [0, 0, 0, 0, 0],
	"isRunning": False
}


In [19]:
def dispCurrentRNState():
	for key, value in RN.items():
		print(key, ": ", value)

In [20]:
def updateRN(processNo, sequenceNumber):
	for key, value in RN.items():
		value[processNo] = max(value[processNo], sequenceNumber)

In [21]:
def executeCS(processForCS):
    print("\n******************************\n")
    print(f"Process {processForCS} executing CS...")
    print('Token owner is: {}'.format(token["token_owner"]))

    time.sleep(10)
    print(f'\nProcess {processForCS} has completed running CS')

    # Process completed CS
    token["isRunning"] = False

    #update LN
    token["LN"][processForCS] = RN[processForCS][processForCS]
    # print(f"Process Completed CS")


    #Check For Outstanding Requests
    # For every site Sj, whose ID is not present in the token queue Q, it appends its ID to Q if RNi[j] = LN[j] + 1 to indicate that site Sj has an outstanding request.
    for index, val in enumerate(RN[token["token_owner"]]):
        # print("Running P: ", runningP)
        if(val == token["LN"][index] + 1 and index != runningP and index not in token["Q"]):
            # outstanding Requests
            print(f'Process {index}\'s request is outstanding, it will be added to Token\'s Queue')
            token["Q"].append(index)
            print(f'Queue: {token["Q"]}')


    #Handing out the token
    if(len(token["Q"]) != 0):
        # pop a process from the queue and give it the token
        poppedPs = token["Q"].pop(0)
        token["token_owner"] = poppedPs
        token["isRunning"] = True
        executeCS(poppedPs)

In [23]:
# def main():
if __name__ == "__main__":
	# print("Press Key E to exit")
	print("Running Main Again")

    # Display Current State of RN Arrays
	print("Current RN Arrays: ")
	dispCurrentRNState()
	print(" ")
	print('Token owner is: {}'.format(token["token_owner"]))

	while True:

		if(token["isRunning"]):
			processes = input("Enter Process Numbers which want to access C.S separated by space (Click N for None): ")
			if(processes != 'N'):
				psList = processes.strip().split(" ")
				print(" ")


				for ps in psList:
					processForCS = int(ps)
					print(f"***** Process {processForCS} *****")
					seqNo = RN[processForCS][processForCS]+1
					# Broadcasting Request

					print(f"Process No.: {processForCS}")
					print(f"Sequence No.: {seqNo}")
					print(f"Broadcasting Request ({processForCS} , {seqNo}) .......")
					time.sleep(2)
					print("Broadcast complete")
					print(" ")

					# Updating RN Arrays
					print("Updating RN Arrays at all process sites")
					updateRN(processForCS, seqNo)
					print("Current RN Arrays: ")
					dispCurrentRNState()
					print(" ")



		else:

			processForCS = int(input("Enter Process No. which wants to access C.S: "))
			seqNo = RN[processForCS][processForCS]+1
            # Broadcasting Request

			print(f"Process No.: {processForCS}")
			print(f"Sequence No.: {seqNo}")
			print(f"Broadcasting Request ({processForCS} , {seqNo}) .......")
			time.sleep(2)
			print("Broadcast complete")
			print(" ")

    		# Updating RN Arrays
			print("Updating RN Arrays at all process sites")
			updateRN(processForCS, seqNo)
			print("Current RN Arrays: ")
			dispCurrentRNState()
			print(" ")

			# Check condition of sending token: RNj[i] = LN[i] + 1
			if(RN[token["token_owner"]][processForCS] == token["LN"][processForCS] + 1):
				# give the token
				print(f"Conditions met, giving token to {processForCS}...")
				token["token_owner"] = processForCS

				# print(f"New Token Owner: {token["token_owner"]}")
				print('Token owner is: {}'.format(token["token_owner"]))

				token["isRunning"] = True
				runningP = processForCS

				thread = threading.Thread(target=executeCS, args=(processForCS, ))
				thread.start()

				print("Main Continuing Running")

		# if str(input()) == 'E' or 'e':
		# 	break


Running Main Again
Current RN Arrays: 
0 :  [0, 1, 0, 0, 0]
1 :  [0, 1, 0, 0, 0]
2 :  [0, 1, 0, 0, 0]
3 :  [0, 1, 0, 0, 0]
4 :  [0, 1, 0, 0, 0]
 
Token owner is: 1
Enter Process No. which wants to access C.S: 1
Process No.: 1
Sequence No.: 2
Broadcasting Request (1 , 2) .......
Broadcast complete
 
Updating RN Arrays at all process sites
Current RN Arrays: 
0 :  [0, 2, 0, 0, 0]
1 :  [0, 2, 0, 0, 0]
2 :  [0, 2, 0, 0, 0]
3 :  [0, 2, 0, 0, 0]
4 :  [0, 2, 0, 0, 0]
 
Conditions met, giving token to 1...
Token owner is: 1

******************************
Main Continuing Running

Process 1 executing CS...
Token owner is: 1
Enter Process Numbers which want to access C.S separated by space (Click N for None): N
Enter Process Numbers which want to access C.S separated by space (Click N for None): 2
 
***** Process 2 *****
Process No.: 2
Sequence No.: 1
Broadcasting Request (2 , 1) .......
Broadcast complete
 
Updating RN Arrays at all process sites
Current RN Arrays: 
0 :  [0, 2, 1, 0, 0]
1 :  [0

ValueError: invalid literal for int() with base 10: 'nhujn'